In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')  # Download VADER lexicon

# Initialize Spacy NLP model
# (Note: Spacy model not used in this code, you may remove it if not needed)
# nlp = spacy.load("en_core_web_sm")

# Load credit cards data
credit_cards_df = pd.read_csv('credit_cards_data.csv')

# Initialize the sentiment analyzer
sid = SentimentIntensityAnalyzer()

def extract_keywords(text):
    """Extract keywords from text."""
    tokens = word_tokenize(text)
    keywords = [word.lower() for word in tokens if word.lower() not in stopwords.words('english')]
    return keywords

def match_keywords_with_cards(keywords, credit_cards_df):
    """Match keywords with credit card features."""
    matched_cards = []
    for _, card in credit_cards_df.iterrows():
        matched_features = []
        for keyword in keywords:
            for column in credit_cards_df.columns:
                if keyword.lower() in str(card[column]).lower():
                    matched_features.append(column)
        matched_cards.append((card['Credit Card'], set(matched_features)))
    return matched_cards

def score_and_rank(matched_cards):
    """Score and rank matched credit cards."""
    scored_cards = [(card, len(features)) for card, features in matched_cards]
    ranked_cards = sorted(scored_cards, key=lambda x: x[1], reverse=True)
    return ranked_cards

def display_recommendations(ranked_cards, top_n=3):
    """Display top N credit card recommendations."""
    print("Top Recommendations:")
    for idx, (card, score) in enumerate(ranked_cards[:top_n], start=1):
        print(f"{idx}. {card} (Score: {score})")

def recommend_credit_cards(customer_input):
    """Recommend credit cards based on customer's input."""
    # Extract keywords from customer's input
    keywords = extract_keywords(customer_input)
    print("Extracted Keywords:", keywords)
    
    # Analyze sentiment of the input text
    sentiment_scores = sid.polarity_scores(customer_input)
    sentiment = 'positive' if sentiment_scores['compound'] >= 0 else 'negative'
    print("Sentiment:", sentiment)
    
    # Match keywords with credit cards
    matched_cards = match_keywords_with_cards(keywords, credit_cards_df)
    print("Matched Cards:", matched_cards)
    
    # Score and rank the matched cards
    ranked_cards = score_and_rank(matched_cards)
    print("Ranked Cards:", ranked_cards)
    
    # Adjust recommendations based on sentiment
    if sentiment == 'positive':
        # Highlight top recommendations more prominently
        display_recommendations(ranked_cards, top_n=5)
    elif sentiment == 'negative':
        # Provide alternative recommendations or address concerns
        print("We're sorry to hear that. Here are some alternative options:")
        # Provide alternative recommendations based on a different criteria
    else:
        # Provide standard recommendations
        display_recommendations(ranked_cards)

# # Example usage
# customer_input = "I want a credit card with cashback rewards and low interest rates."
# recommend_credit_cards(customer_input)


In [ ]:
import pickle

# Define the file path for saving the function
sav_file_path = "recommend_credit_cards.sav"

# Save the function to a .sav file
with open(sav_file_path, 'wb') as f:
    pickle.dump(recommend_credit_cards, f)

print("Function saved successfully to:", sav_file_path)


In [ ]:
def calculate_accuracy(test_data):
    correct_predictions = 0
    total_predictions = 0
    
    for input_text, correct_recommendation in test_data:
        recommendations = recommend_credit_cards(input_text)
        if correct_recommendation in [card for card, _ in recommendations]:
            correct_predictions += 1
        total_predictions += 1
    
    accuracy = correct_predictions / total_predictions
    return accuracy

# Example usage
test_data = [
    ("I want a credit card with cashback rewards and low interest rates.", "Card A"),
    ("I'm looking for a card with travel rewards and no annual fee.", "Card B"),
    # Add more test data here...
]

accuracy = calculate_accuracy(test_data)
print("Accuracy:", accuracy)
